In [346]:
import numpy as np
from __future__ import print_function

In [568]:
def err(size, cov):
    return np.random.multivariate_normal(np.zeros(cov.shape[0]), cov, size=size).T

def mean_kro(x, y):
    C = np.zeros((x.shape[0], y.shape[0]))
    for i in xrange(x.shape[1]):
        C += np.dot(x[:, i:i+1], y[:, i:i+1].T)
    return C/x.shape[1]

def G(u):
    return np.ones(u.shape[0]).dot(u).reshape((1,u.shape[1]))

particles = 100
state_dims = 8
data_dims = 1

u_real = np.ones(state_dims)
y = G(u_real.reshape((state_dims,1))).reshape((data_dims,1))

u = np.random.normal(size=state_dims*particles).reshape((state_dims, particles))

gamma = 0.05
Gamma = gamma*np.eye(data_dims)

n_steps = 5000
h = 1.0 / n_steps

#for j in xrange(n_steps):
j = 0
while j < 5000:
    j += 1
    y_n = y + err(particles, Gamma/h)

    p = G(u)
    
    p_bar = np.average(p, axis=1).reshape((data_dims,1))
    u_bar = np.average(u, axis=1).reshape((state_dims,1))
    
    C_up = mean_kro(u - u_bar, p - p_bar)
    C_pp = mean_kro(p - p_bar, p - p_bar)

    for i in xrange(particles):
        u[:,i] += np.dot(C_up, np.linalg.solve(C_pp + Gamma/h, y_n[:,i] - p[:,i]))
    
    u_hat = u.mean(axis=1)
    y_hat = G(u_hat.reshape((state_dims,1))).reshape((data_dims,1))
    
    if (np.linalg.norm(y-y_hat) < 0.01):
        break
    
print('\nRes:')
u_hat = u.mean(axis=1)
y_hat = G(u_hat.reshape((state_dims,1))).reshape((data_dims,1))

print(j)
print(u_hat)
print(y_hat)
print(y)


Res:
5000
[ 1.01482309  0.97861639  1.29633085 -0.28393651  1.67427961  1.66007831
  0.72184847  0.90108347]
[[ 7.96312368]]
[[ 8.]]
